In [ ]:
!pip install accelerate==0.21.0
!pip install transformers==4.24.0
!pip install transformers datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
# 查看GPU信息
! /opt/bin/nvidia-smi

Mon May 27 13:59:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned.csv')

In [ ]:
# 打乱顺序
df1 = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df1.head()

,ID,label,tweet_content
0,17674,0,更想前想结婚越越结婚难熬辈子碌碌快乐陷入情绪反正想死生活趣趣趣感觉累生活迷茫挺累真太累年月日...
1,32227,0,疯预告放法天全曲听完崩溃默作声崩溃说笑闹社交表面平静糟心事积累程度加菲猫里段话讲加菲猫走丢卖...
2,25159,0,敬入夜支烟丧失兴趣失眠梦早醒评价低消极悲观杀念头力记忆力减退迟缓食欲减退体重减轻性功减退焦虑...
3,18004,0,梦中醒梦梦快乐痛苦骗子转发晚晚熬日子头发越越少码保证线工作员物资没没弟弟前天晚家昨天早里领导...
4,8809,1,心理健康日分享轻度抑郁患发现平时更感劳累果患抑郁段轻躁狂史鲜明前工作时精力充沛感疲劳然稍微工...


In [ ]:
df2=df1

In [ ]:
# choose 2000 rows to retrain the models
df_label_0 = df2[df2['label'] == 0].sample(n=1000, random_state=43)
df_label_1 = df2[df2['label'] == 1].sample(n=1000, random_state=43)

# merge two dataframes
df3 = pd.concat([df_label_0, df_label_1])

In [ ]:
# data =df3 # 2000 rows
data =df1 # all rows

In [ ]:
data.head()

,ID,label,tweet_content
0,15008,0,昨晚 点半 睡点 爸 凶醒 喊 吃 外卖 说点 外卖 睡觉 脸 懵 逼 发现 外卖 哥 送错...
1,5965,1,近 状态 差觉 工作 控力 差 情绪 没 没 睡 精神状态 更差 头痛 欲裂 感觉 眼球 快...
2,14757,0,初 中学 涵笑 精髓 味 确实 艹 确实 确实 草祝 生日快乐 生日 月 日 祝福 分母 味...
3,12609,0,生日 月 日 祝福 臭宝 送 单位 哭 成 傻逼 公里 走 完 慰问 脚 水泡 飞机票 换成...
4,1234,1,转 想想 转转 转草 没 嗑 怪 近 手势 绝太 浪漫 俩 记收 图存 进 备忘录 进 家门...


In [ ]:
# 确保tweet_content列是字符串
data['tweet_content'] = data['tweet_content'].astype(str)


In [ ]:
data.shape

(20650, 3)

In [ ]:
# 将DataFrame转换为Hugging Face的Dataset对象
train_data, test_data = train_test_split(data, test_size=0.1, stratify=data['label'])
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
# 加载预训练的BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"


In [ ]:
# 定义一个tokenizer函数，将tweet_content转换为BERT模型可接受的格式
def tokenize_function(tokenization):
    return tokenizer(tokenization['tweet_content'], padding='max_length', truncation=True)


In [ ]:
# 对数据集进行tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/18585 [00:00<?, ? examples/s]

Map:   0%|          | 0/2065 [00:00<?, ? examples/s]

In [ ]:
# 定义计算评价指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',            # 输出目录
    num_train_epochs=8,                # 训练轮数
    per_device_train_batch_size=8,     # 训练时每个设备的批量大小
    per_device_eval_batch_size=8,      # 评估时每个设备的批量大小
    warmup_steps=500,                  # 预热步数
    weight_decay=0.01,                 # 权重衰减
    logging_dir='./logs',              # 日志目录
    logging_steps=10,                  # 日志记录步数
    evaluation_strategy='epoch'        # 每轮结束后评估模型
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# 定义Trainer对象
# model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)
trainer = Trainer(
    model=model,                       # 训练的模型
    args=training_args,                # 训练参数
    train_dataset=train_dataset,       # 训练数据集
    eval_dataset=test_dataset,         # 测试数据集
    compute_metrics=compute_metrics    # 计算评价指标的函数
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num

In [ ]:
# 开始训练 all rows, 8 epoch,test size=0.1
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592
  Number of trainable parameters = 102269186


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.595700,0.459079,0.820339,0.742836,0.979651,0.844964
2,0.312600,0.320496,0.901211,0.854452,0.967054,0.907273
3,0.289200,0.371103,0.859564,0.791667,0.975775,0.874132
4,0.446200,0.504776,0.768039,0.692415,0.964147,0.805994
5,0.748900,0.442793,0.825182,0.758674,0.953488,0.844998
6,0.382800,0.348157,0.865860,0.992177,0.737403,0.846026
7,0.345900,0.321814,0.884746,0.985330,0.781008,0.871351
8,0.711100,0.685757,0.510896,1.000000,0.021318,0.041746


Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuratio

TrainOutput(global_step=18592, training_loss=0.4191923276673547, metrics={'train_runtime': 14864.2057, 'train_samples_per_second': 10.003, 'train_steps_per_second': 1.251, 'total_flos': 3.91193517109248e+16, 'train_loss': 0.4191923276673547, 'epoch': 8.0})

In [ ]:
# 第三次训练：开始训练 all rows,3 epoch,test size=0.1
trainer.train()

In [ ]:
# 第二次训练：开始训练 all rows,8 epoch,test size=0.3
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14455
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14456
  Number of trainable parameters = 102269186


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.369000,0.318908,0.888458,0.981971,0.791411,0.876453
2,0.275900,0.229477,0.923325,0.916720,0.931224,0.923915
3,0.263300,0.301654,0.897014,0.968381,0.820794,0.888501
4,0.714100,0.702162,0.496045,0.497960,0.985147,0.661535
5,0.754600,0.693264,0.499919,0.499919,1.000000,0.666595
6,0.462200,0.390334,0.871025,0.853756,0.895383,0.874074
7,0.433100,0.392334,0.871186,0.850778,0.900226,0.874804
8,0.202500,0.326131,0.907345,0.929520,0.881498,0.904872


Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6195
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuratio

TrainOutput(global_step=14456, training_loss=0.40354297791907123, metrics={'train_runtime': 12644.4401, 'train_samples_per_second': 9.146, 'train_steps_per_second': 1.143, 'total_flos': 3.04261624418304e+16, 'train_loss': 0.40354297791907123, 'epoch': 8.0})

In [ ]:
# 第一次训练：开始训练 2000 rows,8 epoch,test size=0.1
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 102269186


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.200800,0.439071,0.885000,0.932584,0.830000,0.878307
2,0.157200,0.455705,0.890000,0.943182,0.830000,0.882979
3,0.270800,0.392382,0.910000,0.918367,0.900000,0.909091
4,0.377600,0.367879,0.890000,0.933333,0.840000,0.884211
5,0.099200,0.449124,0.900000,0.916667,0.880000,0.897959
6,0.304300,0.371069,0.915000,0.936842,0.890000,0.912821
7,0.238300,0.352739,0.915000,0.936842,0.890000,0.912821
8,0.214700,0.445657,0.915000,0.936842,0.890000,0.912821


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ID, __index_level_0__, tweet_content. If ID, __index_level_0__, tweet_content are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
The following columns in the evaluation 

TrainOutput(global_step=1800, training_loss=0.22840332890964216, metrics={'train_runtime': 1447.4108, 'train_samples_per_second': 9.949, 'train_steps_per_second': 1.244, 'total_flos': 3788799197184000.0, 'train_loss': 0.22840332890964216, 'epoch': 8.0})

# 防止过度拟合over fitting

In [ ]:
import numpy as np
import random
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback


In [ ]:
import torch

In [ ]:
# 打乱顺序
df1 = df.sample(frac=1).reset_index(drop=True)


In [ ]:
df1.head()

,ID,label,tweet_content
0,17674,0,更想前想结婚越越结婚难熬辈子碌碌快乐陷入情绪反正想死生活趣趣趣感觉累生活迷茫挺累真太累年月日...
1,32227,0,疯预告放法天全曲听完崩溃默作声崩溃说笑闹社交表面平静糟心事积累程度加菲猫里段话讲加菲猫走丢卖...
2,25159,0,敬入夜支烟丧失兴趣失眠梦早醒评价低消极悲观杀念头力记忆力减退迟缓食欲减退体重减轻性功减退焦虑...
3,18004,0,梦中醒梦梦快乐痛苦骗子转发晚晚熬日子头发越越少码保证线工作员物资没没弟弟前天晚家昨天早里领导...
4,8809,1,心理健康日分享轻度抑郁患发现平时更感劳累果患抑郁段轻躁狂史鲜明前工作时精力充沛感疲劳然稍微工...


In [ ]:
# 确保tweet_content列是字符串
df1['tweet_content'] = df1['tweet_content'].astype(str)

In [ ]:
# choose 2000 rows to retrain the models
df_label_0 = df1[df1['label'] == 0].sample(n=1000, random_state=43)
df_label_1 = df1[df1['label'] == 1].sample(n=1000, random_state=43)

# merge two dataframes
df3 = pd.concat([df_label_0, df_label_1])

In [ ]:
# data = df1 # all rows
data = df3 # 2000 rows
# 确保tweet_content列是字符串
data['tweet_content'] = data['tweet_content'].astype(str)

In [ ]:
data.shape

(2000, 3)

In [ ]:
# 将DataFrame转换为Hugging Face的Dataset对象
train_data, test_data = train_test_split(data, test_size=0.1, stratify=data['label'])
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
# 加载预训练的BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 定义一个tokenizer函数，将tweet_content转换为BERT模型可接受的格式
def tokenize_function(tokenization):
    return tokenizer(tokenization['tweet_content'], padding='max_length', truncation=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"


In [ ]:
# 对数据集进行tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# 定义计算评价指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',            # 输出目录
    num_train_epochs=8,                # 训练轮数
    per_device_train_batch_size=8,     # 训练时每个设备的批量大小
    per_device_eval_batch_size=8,      # 评估时每个设备的批量大小
    warmup_steps=500,                  # 预热步数
    weight_decay=0.01,                 # 权重衰减
    logging_dir='./logs',              # 日志目录
    logging_steps=10,                  # 日志记录步数
    evaluation_strategy='epoch',       # 每轮结束后评估模型
    save_strategy='epoch',             # 每轮结束后保存模型
    load_best_model_at_end=True,       # 训练结束后加载最好的模型
    metric_for_best_model='accuracy',  # 用于判断最佳模型的指标
    greater_is_better=True             # 指标越大越好
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# 定义Trainer对象
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)
trainer = Trainer(
    model=model,                       # 训练的模型
    args=training_args,                # 训练参数
    train_dataset=train_dataset,       # 训练数据集
    eval_dataset=test_dataset,         # 测试数据集
    compute_metrics=compute_metrics,   # 计算评价指标的函数
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 早停法，耐心值为3
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num

In [ ]:
# 开始训练
trainer.train()



The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet_content, __index_level_0__, ID. If tweet_content, __index_level_0__, ID are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 102269186


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.730500,0.707569,0.480000,0.489474,0.930000,0.641379
2,0.712100,0.704369,0.500000,0.000000,0.000000,0.000000
3,0.665700,0.698594,0.500000,0.500000,1.000000,0.666667
4,0.711900,0.693565,0.500000,0.000000,0.000000,0.000000
5,0.695300,0.693359,0.500000,0.000000,0.000000,0.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet_content, __index_level_0__, ID. If tweet_content, __index_level_0__, ID are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-225
Configuration saved in ./results/checkpoint-225/config.json
Model weights saved in ./results/checkpoint-225/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet_content, __index_level_0__, ID. If tweet_content, __index_level_0__, ID are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/

TrainOutput(global_step=1125, training_loss=0.7078688413831923, metrics={'train_runtime': 988.3459, 'train_samples_per_second': 14.57, 'train_steps_per_second': 1.821, 'total_flos': 2367999498240000.0, 'train_loss': 0.7078688413831923, 'epoch': 5.0})

In [ ]:
# 保存训练好的模型
trainer.save_model('./best_modelBertBaseChinese2000rows0528')